In [2]:
import networkx as nx
import sqlite3
import pandas as pd

conn = sqlite3.connect('/Users/Admin/Documents/Projects/Kairos/main/data/GTFS.db')

df = pd.read_sql_query('SELECT * FROM gtfs_stops', conn)

In [7]:
df.head(1)

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,stop_code,stop_street,stop_city,stop_region,stop_postcode,stop_country,location_type,parent_station,stop_timezone
0,18848,Clínicas,None,-23.55419,-46.670723,None,None,None,None,None,None,None,None,None,None,None


In [6]:
MG = nx.MultiGraph()

In [ ]:
for i, row in df.iterrows():
    if i > 5: 
        break
    MG.add_node(row['stop_id'])

In [ ]:
MG.nodes